In [30]:
from letta import create_client
from dotenv import load_dotenv
load_dotenv()
client = create_client()

In [31]:
from letta.schemas.block import Block
org_description= "The company is called AgentOS " \
+ "and is building AI tools to make it easier to create " \
+ "and deploy LLM agents."

org_block = Block(name = "company", value = org_description)

In [32]:
from letta.schemas.memory import BasicBlockMemory

class OrgMemory(BasicBlockMemory):
    def __init__(self, persona : str, org_block : Block):
        persona_block = Block(name = "persona", value = persona)
        super().__init__(blocks = [persona_block, org_block])

In [33]:
def read_resume(self, name: str):
    """
    Read the resume data for a candidate given the name

    Args: 
        name (str): Candidate name 

    Returns: 
        resume_data (str): Candidate's resume data 
    """
    import os
    filepath = os.path.join("data", "resumes", name.lower().replace(" ", "_") + ".txt")
    #print("read", filepath)
    return open(filepath).read()

In [34]:
def submit_evaluation(
    self, 
    candidate_name: str, 
    reach_out: bool, 
    resume: str, 
    justification: str
): 
    """
    Submit a candidate for outreach. 

    Args: 
        candidate_name (str): The name of the candidate
        reach_out (bool): Whether to reach out to the candidate
        resume (str): The text representation of the candidate's resume 
        justification (str): Justification for reaching out or not
    """
    from letta import create_client 
    client = create_client()

    message = "Reach out to the following candidate. " \
    + f"Name: {candidate_name}\n" \
    + f"Resume Data: {resume}\n" \
    + f"Justification: {justification}"
    print("eval agent", candidate_name)
    if reach_out:
        response = client.send_message(
            agent_name="outreach_agent", 
            role="user", 
            message=message
        ) 
    else: 
        print(f"Candidate {candidate_name} is rejected: {justification}")

In [35]:
read_resume_tool = client.create_tool(read_resume) 
submit_evaluation_tool = client.create_tool(submit_evaluation)

In [ ]:
skills = "Front-end (React, Typescript), software engineering " \
+ "(ideally Python), and experience with LLMs."
eval_persona = f"You are responsible to finding good recruiting " \
+ "candidates, for the company description. " \
+ f"Ideal canddiates have skills: {skills}. " \
+ "Submit your candidate evaluation with the submit_evaluation tool. "


eval_agent = client.create_agent(
    name="eval_agent", 
    memory=OrgMemory(
        persona=eval_persona, 
        org_block=org_block,
    ), 
    tool=[read_resume_tool.name, submit_evaluation_tool.name]
)